**Importing All the Libraries required**


In [2]:
#importing all the modules
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer #TF-IDF evaluates how relevant a word is to a document in a collection of documents.
from sklearn.linear_model import LogisticRegression
import nltk
nltk.download('punkt')
nltk.download('stopwords')
!python -m spacy download en_core_web_sm


!pip install goslate
!pip install googletrans==3.1.0a0
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
#spacy model,re, Beautifulsoup


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


#**Function to determine Determine the Answer type of the given Telugu Query**

In [3]:
#The model in this function defines the ner tag for given question
def Question_Classification(user_input):
  qa=open("/content/drive/MyDrive/Odqa_telugu_Train_set.txt","a")
  qa.write("    :"+user_input+"\n")
  qa.close()
  f=open("/content/drive/MyDrive/Odqa_telugu_Train_set.txt","r")
  classes=[]
  queries=[]
  new_label=[]
  for line in f:

    line=line.rstrip('\n')
    classes.append((line.split()[0]).split(":")[0])
    lb=(line.split()[0]).split(":")[0]
    #print(lb)
    if(lb=="PERS"):
      new_label.append(1)
    if(lb=="LOCA"):
      new_label.append(2)
    if(lb=="ORGA"):
      new_label.append(3)
    if(lb=="DATE"):
      new_label.append(4)
    if(lb=="TIME"):
      new_label.append(5)
    if(lb=="PERC"):
      new_label.append(6)
    if(lb=="NUMB"):
      new_label.append(7)
    if(lb=="CURR"):
      new_label.append(8)
    queries.append(line[5:])

  vectorizer =TfidfVectorizer(min_df=1)# min_df is 1, which means "ignore terms that appear in less than 1 document". Thus, the default setting does not ignore any terms.
  
  X = vectorizer.fit_transform(classes) #Hence text converted to vectors to process
  Y = vectorizer.fit_transform(queries)

  X_ = Y[:len(new_label)]
  Y_ = new_label[:len(new_label)]
  X_train, X_test, y_train, y_test = train_test_split(X_, Y_, test_size=0.25, random_state=42)
#	print("X_train = ",X_train.shape)
#	print("X_test = ",X_test.shape)


  test=Y[len(queries)-1].toarray()
  


  #nn=MLPClassifier(activation='tanh',solver='sgd',hidden_layer_sizes=(80,50,30,8),random_state=4,alpha=0.1,batch_size=71)#   73.9
  nn = LinearSVC(random_state=0, tol=0.3,loss="squared_hinge",multi_class="crammer_singer")# 76.648
  #nn = LogisticRegression(random_state=0, solver='sag', multi_class='multinomial')# 73.9
  

	

  nn.fit(X_train,y_train)
  pred2=nn.predict(X_test)


  hits=0.00
  for i in range(0,len(y_test)):
    if y_test[i]==pred2[i]:
      hits=hits+1
  print("The accuracy is ",((hits/len(y_test))*100.0))
  
  
  
	

  result=nn.predict(test)
  
  if(result==1):
    ans_type.append("PERSON")

  elif(result==2):
    ans_type.append("LOCATION")

  elif(result==3):
    ans_type.append("ORGANIZATION")

  elif(result==4):
    ans_type.append("DATE")

  elif(result==5):
    ans_type.append("TIME")

  elif(result==6):
    ans_type.append("PERCENTAGE")
  
  elif(result==7):
    ans_type.append("NUMBER")
  else:
    ans_type.append("CURRENCY")
  return(ans_type[0])
user_input_telugu="“అడ్వెంచర్స్ ఆఫ్ టామ్ సాయర్” రచయిత ఎవరు?" #input("enter query")
ans_type=[]
Question_Classification(user_input_telugu)


The accuracy is  75.54945054945054


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


'PERSON'

#**Language Translation section**

In [ ]:
#TO TRANSLATE LIST OF QUESTIONS AND LIST OF ANSWERS FROM TELUGU TO ENGLISH
def translate(quest_list,ans_list):
  from googletrans import Translator
  translater=Translator()
  out_quest=[] #tranlated list
  out_ans=[]
  for quest in quest_list:
    translated=translater.translate(quest,dest="en")
    out_quest.append(translated.text)
  for ans in ans_list:
    translated=translater.translate(ans,dest="en")
    out_ans.append(translated.text)
  return out_quest,out_ans
(Target_Questions,Target_Answers)=translate(["భారత దేశ ప్రధాని ఎవరు?","అడ్వెంచర్స్ ఆఫ్ టామ్ సాయర్” రచయిత ఎవరు?"],["మోడీ"])
print(Target_Questions,"and",Target_Answers)

['Who is the Prime Minister of India?', "Who is the author of 'Adventures of Tom Sawyer'?"] and ['Modi']


In [ ]:
#TO TRANSLATE SINGLE QUESTION FROM TELUGU TO ENGLISH
def translate_2(to_translate, to_langage="auto", langage="auto"):
        import goslate
        gs = goslate.Goslate()
     #   print(gs.translate(to_translate, 'en'))
        result = gs.translate(to_translate,'en')
        return result
translate_2("భారత దేశ ప్రాధాని ఎవరు?")


'Who is the Prime of India?'

In [ ]:
#To translate a FINAL ANSWER from English to Telugu
def translate_eng_to_tel(ans):
  from googletrans import Translator
  translater=Translator()
  out=translater.translate(ans,dest="te")
  print(out.text)
  return out.text
print(translate_eng_to_tel("country"))

దేశం
దేశం


#**Translating whole File which is in Telugu to English**

In [ ]:
def translate_file():
  t = open("/content/drive/MyDrive/odqa_telugu_testset.txt","r").readlines()
  print(len(t))
  Target_Questions_telugu = []#All the Questions(which are in telugu) in test set will be copied into this list
  Target_Answers_telugu	 = []#All the corresponding Answers are copied into this list

  #copying the contents in the test set to above two lists
  for line in t:
    line=line.strip()
    line=line.split("@@@")
    Target_Questions_telugu.append(line[0].strip())
    Target_Answers_telugu.append(line[1].strip())
  
  #Now we translate whole list of Questions and Answers into which are in Telugu to English using below code
  (Target_questions,Target_answers)=translate(Target_Questions_telugu,ans_list)# translate funcion defined above

  #Now we store Tranlsated questions and Answers which are in english in a new file named "odqa_telugu_testset.te.en.text"
  f = open("/content/drive/MyDrive/odqa_telugu_testset.te_en.txt","w")
  for i in range(0,len(Target_questions)):
    f.write(Target_questions[i]+" "+Target_answers[i]+"\n")
  f.close()
#Now we are ready with test set in telugu for Question Classificationa and test set english for web scraping


  

**Performing NER tagging for Extracted text**
#This function outputs possible answers list

In [ ]:
def ner_tag(sents,tag):
  import spacy 
  from spacy import displacy
  if(tag=="LOCATION"):
    tag="GPE"
  if(tag=="NUMBER"):
    tag="CARDINAL"
  if(tag=="ORGANIZATION"):
    tag="ORG"
  if(tag=="CURRENCY"):
    tag="MONEY"

  

  # Load SpaCy model
  nlp = spacy.load("en_core_web_sm")
  
  import pandas as pd
  
  doc = nlp(sents)

  entities = []
  labels = []
  position_start = []
  position_end = []

  for ent in doc.ents:
    entities.append(ent)
    labels.append(ent.label_)
    position_start.append(ent.start_char)
    position_end.append(ent.end_char)
    #print(entities, labels)
  
  ans=[]  
  df = pd.DataFrame({'Entities':entities,'Labels':labels}) #,'Position_Start':position_start, 'Position_End':position_end})
  for i in range(0,len(labels)):
    if labels[i]==tag:
      ans.append(entities[i])
  print(df)

  

    
  print("possible answers list is",ans)
  #print("final answer is",ans[0])
  #final_ans=ans[0]
      



 
  return ans
#ner_tag("narendra modi on 2-03-2021  gave 1 billion dollars, RS.900, $351 AND 500 dollars supreme court is prime minister  Rs 540 crore of Lom Turkish Togo  lira of india","ORGANIZATION") 
#ner_tag("Lome 1650","LOCATION")
ner_tag("\n Mark  Twain, MARK TWAIN AND  Narendra modi money 500 dollars should NOT exceed INR 4000 USD 100 per annum.","CURRENCY")

                Entities   Labels
0             (Narendra)      ORG
1  (money, 500, dollars)    MONEY
2                  (INR)      ORG
3       (4000, USD, 100)  PRODUCT
possible answers list is [money 500 dollars]


[money 500 dollars]

#**Funciton to find Sequence Matching SCORE for two strings**

In [ ]:

def similarity(str1,str2):
  str1=str1.lower()
  str2=str2.lower()
  #print(str1,str2)
  
  from difflib import SequenceMatcher
  score=SequenceMatcher(lambda x: x==" ",str1,str2).ratio()
  
  #print(score)
  return score
similarity("venu","Vanu")



0.75

#**Special NER tagger for CURRENCY type**

In [ ]:
def curr_ner(words_list):
  #print(words_list)
  ans_list=[]
  
  train=['Turkish lira','Turkish','lira','Birr', 'peso', 'Metical', 'Mark', 'Balboa', 'Ruble', 'Euro', 'Krona', 'Lake', 'Colon', 'Euro', 'Sudanese pound', 'Kina', 'Corona', 'Turkish lira', 'Deutsche Mark', 'Malaysian Dollar', 'Kuwaiti Dinar', 'Inti Soul', 'Drachma', 'pound', 'Bolivar', 'Real', 'New Shekel', 'peso', 'Colon', 'pound', 'Tugrick', 'French Frank', 'American Dollar', 'Shilling', 'Rs. 1000', 'American Dollar', 'Rs 2 crore', 'Rs 540 crore', 'Rs. 115', 'Rs.5,000','₹','₹1234567890','750-million-dollar', '50 pence', 'Rs.650 crore', '1 billion', 'USD 75 million', 'Rs.50,000 crores', 'Rs 2 trillion', 'Rs 2 trillion', 'Euro', '1,01,500', '1,8292 crores', 'USD 15 million']
  #print(train)
  i=0
  for word in words_list:
    for word_in_train in train:
      sc=similarity(word_in_train,word)
      #print(word_in_train,word,sc)
      if(sc>=0.7):
        ans_list.append(word)
    i+=1
  print("Possible answers list is ",ans_list)
  return (ans_list)
curr_ner(['Birr',"nothing"])

Possible answers list is  ['Birr']


['Birr']

#**Preprocessing Extracted Data (so that spacy ner tagger works well on extracted data)**

In [ ]:
# Extracted data not having space between the words. so it is becoming difficult for spacy ner tagger to identify the tags.
#This function takes that unspaced extracted data as input and outputs data with spaces between the words
import re #For Regular expressions.

def editing(phrase):
  
  #print(phrase)
  new_phrase=""
  j=0
  pattern=re.compile(r'[A-Z][a-z]+')
  
  list=[]
 
  new_phrase=phrase
  k=0
  for i in pattern.finditer(phrase):
    #print("true")
    #print(new_phrase[i.start()-1+k:i.start()+k])
    if(new_phrase[i.start()-1+k:i.start()+k]!=" "):
      new_phrase=new_phrase[0:i.start()+k]+" "+new_phrase[i.start()+k:]
      k+=1
  print("preprocessed extracted text is:::",new_phrase)
  return new_phrase
find("YES! ItIf1895Mark Twain  1650 AD worked ")



preprocessed extracted text is::: YES! It If1895 Mark Twain  1650 AD worked 


'YES! It If1895 Mark Twain  1650 AD worked '

## **Main code starts here**

In [ ]:
#MAIN CODE

from bs4 import BeautifulSoup
import requests, lxml, os
from nltk import sent_tokenize
from nltk import word_tokenize

headers = {
    'User-agent':
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

proxies = {
  'http': os.getenv('HTTP_PROXY')
}

t = open("/content/drive/MyDrive/odqa_telugu_testset.txt","r").readlines()
print(len(t))
Target_Questions_telugu = []#All the Questions(which are in telugu) in test set will be copied into this list
Target_Answers_telugu	 = []#All the corresponding Answers((which are in telugu) are copied into this list

#copying the contents in the test set to above two lists
for line in t:
  line=line.strip()
  line=line.split("@@@")
  Target_Questions_telugu.append(line[0].strip()[5:])
  Target_Answers_telugu.append(line[1].strip())

t2 = open("/content/drive/MyDrive/odqa_telugu_testset.te.en.txt","r").readlines()
print(len(t2))
Target_Questions = [] #This list will have all the Question in ENGLISH language
Target_Answers	 = [] #This list will have all the Answers in ENGLISH language

for line in t2:
  line=line.strip()
  line=line.split("?")
  Target_Questions.append(line[0].strip()[5:]+"?")#copying only question not the ans type whih is at 0 to 4 indexes
  Target_Answers.append(line[1].strip())


#t = open("/content/drive/MyDrive/odqa_telugu_testset_english.txt","r").readlines()

n=int(input("no of questions to be searched"))
hit=0
partial=0
partial_2=0
help=0 #int(input("ENTER starting position of tag(like 250 for curr,200 for number,150 for date,100 for org,50 for loc,0 for pers"))

help2=""#input("enter helping tag")
#(Target_Questions,Target_Answers)=translate(Target_Questions_telugu[0:n],Target_Answers_telugu[0:n]) # Test questions and Test answers are converted from telugu to english
#print("checking for conversion to english from telugu")
print(Target_Questions[0:n])
#print(Target_Answers[0:n])

z=0
count=0
final_answer_list=[]
for z in range(0,n):
  if(z>=0 and z<50):#These conditions useful to find Quesion classification accuracy
    help2="PERSON"
  if(z>=50 and z<100):
    help2="LOCATION"
  if(z>=100 and z<150):
    help2="ORGANIZATION"
  if(z>=150 and z<200):
    help2="DATE"
  if(z>=200 and z<250):
    help2="NUMBER"
  if(z>=250 and z<300):
   help2="CURRENCY"  
  z=z+help

  print("\nsearching for question",z+1,"..............,")
  print("Question: ",Target_Questions[z])

  
  user_input_telugu=Target_Questions_telugu[z]
  user_input=Target_Questions[z]    #input("Enter your Query in telugu ")
  
  ans_type=[]
  tag=Question_Classification(user_input_telugu)
  print(tag)
  
  if(tag==help2):#Counting number of correct Question Classification. "count" variable used to find accuracy of QC at the end of this code
    count+=1
  



  user_input_english=Target_Questions[z] #input('search in english: ')
  #user_input_english=translate_2(user_input_telugu2)






  #user_input_english=translate_2(user_input_telugu2)
  google_search=requests.get("https://www.google.com/search?q="+user_input_english, headers=headers, proxies=proxies)
  #google_search= requests.get('https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q='+user_input_english)
  #print(google_search.text)
  soup=BeautifulSoup(google_search.content,'html.parser')

  #print("########################################################################################################")
  #print("text file is",soup.text)
  snippet=""
  for s in soup.findAll(id='res'):
    #print(s.)
    print('true')
    
    s=s.text.replace("Search ResultsFeatured snippet from the web"," ").split("(")[0]
    ns=s.replace("Search ResultsWeb results","no snippet: ")
    data=ns
  print(data)
    
  final_data=find(data)


  try:
    if(tag=='CURRENCY'):
      print("OKKKKK")
      words_list=word_tokenize(final_data)
      
      ans=curr_ner(words_list)
    else:
      ans=ner_tag(final_data,tag)
    #print(df)
    
    

  except:
    ans=["NO ANSWER"]
    
    pass
  k=1
  try:
    final_ans=ans[0]
  
    while(user_input_english.editing(str(final_ans))!=-1):
      
      final_ans=str(ans[k])
      k+=1
  except:
    final_ans="NO ANSWER"
    

  print('final answer is', final_ans)
  print('final answer in telugu is ',translate_eng_to_tel(final_ans))
  print("Actual answer is",Target_Answers[z])
  final_answer_list.append(str(final_ans))

  similar={}
  target_ans_words=set()
  final_ans_words=set()
  target_ans_words=set(word_tokenize(Target_Answers[z]))
  final_ans_words=set(word_tokenize(str(final_ans)))
  rvector=set()
  rvector = target_ans_words.union(final_ans_words) 
  v1=[] #for target answers
  v2=[]
  
  for w in rvector:
    if w in target_ans_words:
      v1.append(1)
    else:
      v1.append(0)
    if w in final_ans_words:
      v2.append(1)
    else:
      v2.append(0)
  c=0
  j=0
  try:
    for j in range(len(rvector)):
      c+=v1[j]*v2[j]
    v1_square=[v**2 for v in v1]
    v2_square=[v**2 for v in v2]
      
    cosine2=c/(float((sum(v1_square)*sum(v2_square))**0.5))
    print("cosine similarity is ",cosine2)
     
  except:
    pass
  score=similarity(Target_Answers[z],str(final_ans))
  print("Sequence matcher score is ",score)
  if cosine2==1.0 or score==1.0:
    hit+=1
  if cosine2>=0.7 or score>=0.7:
    partial+=1
  if cosine2>=0.4 or score>=0.5:
    partial_2+=1
  print("hits are ",hit,partial,partial_2)
  print("#################################################")
        



accuracy= hit*100/n
print("actual answers list is,",Target_Answers[0:n])
print("answers that I got are",final_answer_list)
print("Exact match accuracy for",n,"no of questions is ",accuracy)

#print("no of partial correct ans are ",partial)
partial_accuracy=partial*100/n
print("partial match accurccy is for ",n,"no of questions is ",partial_accuracy)

#print("no of partial_2(cosine>=0.5) are ",partial_2)
partial_2_accuracy=partial_2*100/n
print("partial_2 accuracy(cosinesimilarity>=0.5) is for ",n,"no of questioons is ",partial_2_accuracy)
#print("total count is",count)
print("Quesiton classification accuracy is ",(count*100)/n)



  

300
300
no of questions to be searched3
[" Who is the author of 'Adventures of Tom Sawyer'?", " Who is the author of 'Kavya Narthaki'?", " Who is the author of 'Don Juan'?"]

searching for question 1 ..............,
Question:   Who is the author of 'Adventures of Tom Sawyer'?
PERSON


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


true
Search ResultsThe Adventures of Tom Sawyer/AuthorMark Twain
 Search Results The Adventures of Tom Sawyer/ Author Mark Twain
        Entities  Labels
0  (Mark, Twain)  PERSON
possible answers list is [Mark Twain]
final answer is Mark Twain
మార్క్ ట్వైన్
final answer in telugu is  మార్క్ ట్వైన్
Actual answer is Mark Twain
cosine similarity is  1.0
Sequence matcher score is  1.0
hits are  1 1 1
#################################################

searching for question 2 ..............,
Question:   Who is the author of 'Kavya Narthaki'?
PERSON


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


true
no snippet: Changampuzha Krishna Pillai - Wikipediahttps://en.wikipedia.org › wiki › Changampuzha_Krish...https://en.wikipedia.org › wiki › Changampuzha_Krish...Occupation, Poet, writer. Language, Malayalam. Alma mater, Maharaja's College, Ernakulam. Period, 1931–1948. Genre, Romantic. Notable works, Ramanan. Spouse, Sreedevi Amma. Children, Sreekumar, Ajitha, Jayadev, Lalitha. Changampuzha Krishna Pillai 
no snippet: Changampuzha Krishna Pillai - Wikipediahttps://en.wikipedia.org › wiki › Changampuzha_ Krish...https://en.wikipedia.org › wiki › Changampuzha_ Krish... Occupation, Poet, writer. Language, Malayalam. Alma mater, Maharaja's College, Ernakulam. Period, 1931–1948. Genre, Romantic. Notable works, Ramanan. Spouse, Sreedevi Amma. Children, Sreekumar, Ajitha, Jayadev, Lalitha. Changampuzha Krishna Pillai 
                             Entities    Labels
0     (Changampuzha, Krishna, Pillai)    PERSON
1  (Krish...https://en.wikipedia.org)  CARDINAL
2                         (M

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


true
 Lord ByronLord Byron derived the character, but not the story, from the Spanish legend of Don Juan....Don Juan 
 Lord Byron Lord Byron derived the character, but not the story, from the Spanish legend of Don Juan.... Don Juan 
               Entities  Labels
0  (Byron, Lord, Byron)  PERSON
1             (Spanish)    NORP
2           (Don, Juan)  PERSON
3           (Don, Juan)  PERSON
possible answers list is [Byron Lord Byron, Don Juan, Don Juan]
final answer is Byron Lord Byron
బైరాన్ లార్డ్ బైరాన్
final answer in telugu is  బైరాన్ లార్డ్ బైరాన్
Actual answer is Lord Byron
cosine similarity is  1.0
Sequence matcher score is  0.38461538461538464
hits are  3 3 3
#################################################
actual answers list is, ['Mark Twain', 'Changampuzha Krishna Pillai', 'Lord Byron']
answers that I got are ['Mark Twain', 'Changampuzha Krishna Pillai', 'Byron Lord Byron']
Exact match accuracy for 3 no of questions is  100.0
partial match accurccy is for  3 no of questions